# Introdução a Ciência de Dados
## Exercicios em Sala
https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset?resource=download

Utilizando a base "realtor-data.csv" gerar as seguintes features:
 - maior tamanho de casa por cidade. 
 - menor tamanho de casa por cidade. 
 - tamanho de casa médio por cidade. 
 - porcentagem de missing values do tamanho de casa por cidade. 


In [1]:
import pandas as pd
import numpy as np

In [2]:
realtor = pd.read_csv("realtor-data.csv")
realtor.head(3)

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,NaN,105000.0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,NaN,80000.0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,NaN,67000.0


In [34]:
realtor['hs_missing'] = np.where(realtor['house_size'].isnull(),1,0)
features1 = (realtor
            .groupby(['city','state'])
            .agg({"house_size":['max','min','mean'],'hs_missing':'mean'})
            .reset_index())
features1.columns = ['city','state','hs_min','hs_max','hs_mean','hs_missing']
features1.head(2)

,city,state,hs_min,hs_max,hs_mean,hs_missing
0,Acton,Massachusetts,99999.0,918.0,4940.695652,0.157509
1,Adams,Massachusetts,9516.0,640.0,2332.830078,0.316422


 - razão do price máximo pelo acre_lot mínimo por cidade. 
   - quando acre_lot = 0, adicionar o missing simbólico -1 

In [38]:
features2 = (realtor
            .groupby(['city','state'])
            .agg({"price":'max','acre_lot':'min'})
            .reset_index())
features2.columns = ['city','state','price_max','acre_lot_min']

features2['rz_price_acre_lot'] = (features2['price_max']/features2['acre_lot_min']).round(1)

#quando acre_lot = 0, adicionar o missing simbólico -1 
features2.loc[features2['acre_lot_min']==0,'rz_price_acre_lot'] = -1

features2 = features2.drop(columns=['price_max','acre_lot_min'])

features2.head(2)

,city,state,rz_price_acre_lot
0,Acton,Massachusetts,91666666.7
1,Adams,Massachusetts,24750000.0


In [39]:
#join das features
features = features1.merge(features2,on=['city','state'],how='left')
features

,city,state,hs_min,hs_max,hs_mean,hs_missing,rz_price_acre_lot
0,Acton,Massachusetts,99999.0,918.0,4940.695652,0.157509,91666666.7
1,Adams,Massachusetts,9516.0,640.0,2332.830078,0.316422,24750000.0
2,Adjuntas,Puerto Rico,1527.0,920.0,1214.303030,0.000000,1312500.0
3,Agawam,Massachusetts,3200.0,676.0,1675.292230,0.070644,7090000.0
4,Aguada,Puerto Rico,6000.0,600.0,2139.268116,0.471264,142857142.9
...,...,...,...,...,...,...,...
577,Worcester,Massachusetts,14532.0,456.0,2469.285381,0.087511,44875000.0
578,Worthington,Massachusetts,14240.0,2180.0,6878.363636,0.629213,1081227.4
579,Wrentham,Massachusetts,3872.0,1044.0,2290.941176,0.227273,7500000.0
580,Yabucoa,Puerto Rico,4902.0,849.0,1634.038462,0.434783,7600000.0
